In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

In [ ]:
data = pd.read_csv("J:/Data science/data/insurance data/insurance.csv")
data.head()

In [ ]:
df = data.copy()

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
correlation_matrix = df.corr(numeric_only=True)
correlation_matrix

In [ ]:
plt.figure(figsize=(4,3))
sns.heatmap(correlation_matrix, annot=True)
plt.show()

In [ ]:
df.dtypes

In [ ]:
# Convert 'charges' column to float64
df['charges'] = pd.to_numeric(df['charges'], errors='coerce')

In [ ]:
df.charges.dtype

In [ ]:
text_columns = df.select_dtypes(include=['object']).columns
text_columns

In [ ]:
def val_cont_df(df):
    # Identify text (categorical) columns
    text_columns = df.select_dtypes(include=['object', 'category']).columns
    # List to store value_counts DataFrames
    value_counts_dfs = []
    
    # Iterate over text columns and compute value_counts
    for col in text_columns:
        value_counts = df[col].value_counts().reset_index()
        value_counts.columns = ['Value', 'Count']
        value_counts['Column'] = col
        value_counts_dfs.append(value_counts)

    # Concatenate all DataFrames
    result_df = pd.concat(value_counts_dfs, ignore_index=True)
    return result_df
    


In [ ]:
result_df = val_cont_df(df)
result_df

### Edit sex column

In [ ]:
# Mapping dictionary
gender_map = {
    'male': 'M',
    'man': 'M',
    'female': 'F',
    'woman': 'F',
}
df['sex'] = df['sex'].map(gender_map)

In [ ]:
df.sex.value_counts()

In [ ]:
val_count = val_cont_df(df)
val_count

In [ ]:
df.region.dtype

In [ ]:
# Define the function
def correct_region(region):
    if isinstance(region, str):
        return region.lower()
    else:
        return np.nan  # Handle missing/invalid values


In [ ]:
# Apply the function
df['region'] = df['region'].apply(correct_region)

In [ ]:
df.region.value_counts()

In [ ]:
valcount = val_cont_df(df)
valcount

In [ ]:
df.shape

In [ ]:
df.isna().sum()

## impute age, bmi,charges with mean

In [ ]:
# Impute float variables with mean
float_cols = ['age', 'bmi', 'charges']
df[float_cols] = df[float_cols].fillna(df[float_cols].mean())

## Impute Discrete Variable (children) with Median

In [ ]:
# Impute discrete variable with median
df['children'] = df['children'].fillna(df['children'].median())

### Encode Categorical Variablesm by OneHotEncoder for nominal data

In [ ]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder

In [ ]:
df.columns

In [ ]:
# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit and transform categorical columns
encoded_data = encoder.fit_transform(df[['sex', 'smoker', 'region']])

# Create a DataFrame from the encoded data
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['sex', 'smoker', 'region']))

# Drop original categorical columns and concatenate encoded columns
df2 = df.drop(columns=['sex', 'smoker', 'region'])
df2 = pd.concat([df2, encoded_df], axis=1)


In [ ]:
df2.head(2)

In [ ]:
df2.shape

In [ ]:
df.head(2)

In [ ]:
df.shape

In [ ]:
df2.head()

## Impute Categorical Variables with KNN

In [ ]:
# Apply KNN Imputation
imputer = KNNImputer(n_neighbors=3)
df_imputed = pd.DataFrame(imputer.fit_transform(df2), columns=df2.columns)

In [ ]:
df_imputed.head()

In [ ]:
df_imputed.isna().sum()

In [ ]:
# # Inverse transform one-hot encoded columns
# decoded_data = encoder.inverse_transform(df_imputed[encoder.get_feature_names_out(['sex', 'smoker', 'region'])])

# # Add decoded columns back to the DataFrame
# df_imputed[['sex', 'smoker', 'region']] = decoded_data

# # Drop one-hot encoded columns
# df_imputed2 = df_imputed.drop(columns=encoder.get_feature_names_out(['sex', 'smoker', 'region']))

In [ ]:
df_imputed.head()

In [ ]:
df_imputed.duplicated().sum()

In [ ]:
df_imputed.drop_duplicates(inplace=True)

In [ ]:
df_imputed.columns

In [ ]:
df_imputed.drop(columns=['sex_nan','smoker_nan','region_nan'],inplace=True)

In [ ]:
df_imputed.columns

In [ ]:
df_imputed.shape

In [ ]:
df_imputed.to_csv('Insurance_data_processed_imputed.csv',index=False)

In [ ]:
dd = pd.read_csv('Insurance_data_processed_imputed.csv')

In [ ]:
dd.head()

In [ ]:
dd.columns

In [ ]:
dd['sex'] = dd.apply(lambda row:'F' if row['sex_F']== 1.0 else 'M', axis=1)
dd['smoker'] = dd.apply(lambda row:'no' if row['smoker_no']==1.0 else 'yes', axis=1)
dd['region'] = dd.apply(lambda row:'northeast' if row['region_northeast'] == 1.0 else
                                   'northwest' if row['region_northwest'] == 1.0 else
                                   'southeast' if row['region_southeast'] == 1.0 else
                                   'southwest', axis=1)

In [ ]:
dd = dd.drop(columns=['sex_F', 'sex_M', 'smoker_no', 'smoker_yes', 
                      'region_northeast', 'region_northwest', 'region_southeast', 'region_southwest'])

In [ ]:
dd.head()

In [ ]:
dd.isna().sum()

In [ ]:
dd.to_csv('Insurance_data_processed_imputed.csv', index=False)

In [3]:
final_data = pd.read_csv('Insurance_data_processed_imputed.csv')
final_data.head()

,age,bmi,children,charges,sex,smoker,region
0,19.0,27.900,0.0,16884.924000,F,yes,southwest
1,18.0,33.770,1.0,1725.552300,M,no,southeast
2,28.0,33.000,3.0,13424.206937,M,no,southeast
3,33.0,22.705,0.0,13424.206937,M,no,northwest
4,32.0,28.880,0.0,13424.206937,M,no,northwest


In [4]:
final_data[final_data.duplicated()]

,age,bmi,children,charges,sex,smoker,region
233,35.214623,30.56055,1.0,13424.206937,M,no,southwest
247,35.214623,30.56055,1.0,13424.206937,M,no,southeast
559,35.214623,30.56055,1.0,13424.206937,M,no,southwest
1048,35.214623,30.56055,1.0,13424.206937,M,yes,northeast
1113,35.214623,30.56055,1.0,13424.206937,M,yes,southwest
1248,35.214623,30.56055,1.0,13424.206937,M,yes,southwest
1304,35.214623,30.56055,1.0,13424.206937,M,no,southwest


In [7]:
final_data.duplicated().sum()

np.int64(7)

In [8]:
final_data.drop_duplicates(inplace=True)

In [9]:
final_data.duplicated().sum()

np.int64(0)

In [10]:
final_data.to_csv('Insurance_data_processed_imputed.csv',index=False)